#### Google Official Documents
https://developers.google.com/drive/api/v3/quickstart/python<br>
https://developers.google.com/drive/api/v3/manage-downloads<br>
https://developers.google.com/drive/api/v3/reference/files/list<br>
https://developers.google.com/drive/api/v3/search-files<br>

#### STEP1: 0Authetication
Credit for the majority of the code below goes to the article from Medium (link below)<br>
https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c

In [1]:
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
obj = lambda: None # this code allows for an empty class
auth = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in auth.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('token_google_drive.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

#### STEP2: Search for files and create list of files to download
look below for supported mimeType<br>
https://developers.google.com/drive/api/v3/mime-types

In [8]:
#### Search for Files

from googleapiclient.discovery import build

page_token = None
drive_service = build('drive', 'v3', credentials=creds)
while True:
    name_list = []
    id_list = []
    response = drive_service.files().list(q="mimeType='text/csv' and name contains 'SRP_final'",
                                          ##### MAKE SURE YOU ADJUST THIS!!!!
                                          # AND for more restrictive conditions, OR for alternating conditions
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        name = file.get('name')
        id_ = file.get('id')
        
        #name and id are strings, so create list first before creating a dictionary
        name_list.append(name)
        id_list.append(id_)
        
        #also you need to remove ":" in name_list or you cannot download files - nowhere to be found in the folder!
        name_list = [word.replace(':','') for word in name_list]
    
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break

#### Create dictionary using name_list and id_list
zipobj = zip(name_list, id_list)
temp_dic = dict(zipobj)

#clean up the objects
del name_list
del id_list
del zipobj

In [9]:
temp_dic #shows the entire dictionary of files to donwload

{}

#### STEP 2.5: Before downloading files, create a folder dedicated for raw data of last month
https://stackabuse.com/creating-and-deleting-directories-with-python/

In [12]:
# CHECK CURRENT DIRECTORY

import os
# code below shows current directory
os.getcwd()

'C:\\Users\\skato\\Desktop\\Regional Report_Automation\\201910'

In [15]:
# CREATE A NEW DIRECTORY

# define the name of the directory to be created 
path = "C:\\Users\\skato\\Desktop\\Regional Report_Automation\\201910" ### MAKE SURE YOU ADJUST THIS!!!!
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s faileds" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory C:\Users\skato\Desktop\Regional Report_Automation\201910 faileds


In [11]:
# CHANGE ACTIVE DIRECTORY

# change working directory to the folder just created
os.chdir('C:\\Users\\skato\\Desktop\\Regional Report_Automation\\201910')

#### STEP3: Download files

In [14]:
import io
from googleapiclient.http import MediaIoBaseDownload

for i in range(len(temp_dic.values())):

    file_id = list(temp_dic.values())[i]
    v = list(temp_dic.keys())[i]
    request = drive_service.files().get_media(fileId=file_id)
    
    # replace the filename and extension in the first field below
    fh = io.FileIO(v, mode='wb') #only in Windows, writing for binary is specified with wb
    downloader = MediaIoBaseDownload(fh, request)
    
    status, done = downloader.next_chunk()
    status_complete = int(status.progress()*100)
    print(f'Downloading {list(temp_dic.keys())[i]} is {int(status.progress()*100)}% complete')
    
fh.close()
print(f'{len(list(temp_dic.keys()))} files')

26 files


In [27]:
# #Code for a single file 1st in the list

# file_id = list(temp_dic.values())[0]
# v = list(temp_dic.keys())[0]
# request = drive_service.files().get_media(fileId=file_id)
# # replace the filename and extension in the first field below
# fh = io.FileIO(v, mode='w')
# downloader = MediaIoBaseDownload(fh, request)
# done = False

# while done is False:
#     status, done = downloader.next_chunk()
#     status_complete = int(status.progress()*100)
#     print(f'Download of {len(temp_dic.values())} files, {int(status.progress()*100)}%')

#### Action Items
(1) SOLVED: Put multiple query conditions<br>  e.g. q="mimeType='text/csv' and name contains 'RR' and name contains '20190901'" <br><br>
(2) SOLVED Find files<br> if filename contains colon, <b>" : "</b>, python does not recognizes it and nothing is stored: solution is to remove :<br><br>
(3) SOLVED Place all the filenames and ids into dictionary so I can loop and automate file downloads<br><br>
(4) SOLVED Iterate Download Files through i in temp_dic
>Part 1: for x loop top
> > a. Convert dictionary values into strings (str)<br>
> > b. but you cannot range(temp_dic.values()) because temp_dic.values() is not integer<br>
> > c. but you cannot len(temp_dic.values()) because i loop requires range, not integer<br>
> > d. for i in range(len(temp_dic.values())):

>Part 2: Turn dictionary values into strings
>> a. you need to turn values into list and let it pick index 'i' <br>
file_id = list(temp_dic.values())[i]

(5) DEBUG SOLVED: files downloaded are all empty --> take out 'while donw is False' & 'done = False'<br>

(6) Specify directory to which you download raw data